In [1]:
import numpy as np
import pandas as pd
import os
import re
import pyedflib

from scipy.signal import butter, lfilter
from pyentrp import entropy as ent
from scipy.signal import welch
from scipy.signal import boxcar
from scipy.stats import variation
from scipy.stats import skew
from scipy.stats import kurtosis
from scipy.stats import describe

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns


In [5]:
def generateEEGdataDF(file, fft_seg_length):
#this function loads the data files, and parses the data into an appropriate dataframe
    
    #open the file, catch exception if file is already open
    try:
        f = pyedflib.EdfReader(file)
    except OSError:
        print('File already open')
        f._close()
    
    #load relevent parameters into df
    numChannels = f.signals_in_file
    channelName=[]
    channelSamples=[]
    Fs=[]
    traces=[]
    for channel in range(numChannels):
        channelName.append(f.getLabel(channel))
        channelSamples.append(f.getNSamples()[channel])
        Fs.append(f.getSampleFrequency(channel))
        traces.append(f.readSignal(channel))
        
    EEGdata=pd.DataFrame([channelName,channelSamples,Fs,traces],index=['channelName','channelSamples','Fs','traces'])
    
    #get engineered features from loaded data, add to existing df
    segmentedFFT(EEGdata,fft_seg_length)
    
    #remember to close the file!
    f._close()
    
    return EEGdata.transpose()

In [3]:
def segmentedFFT(EEGdata,seg_length):
#this function calculates Short Fourier Transform
#length specified (seg_length)    
    for i in EEGdata.columns:
        
        samples_per_segment = EEGdata.loc['channelSamples'][i]/(EEGdata.loc['Fs'][i]/seg_length)
        tot_segments = np.floor(EEGdata.loc['channelSamples'][i]/samples_per_segment).astype(np.int)
        
        #calculate the start and end times of each segment
        startSamples = [np.int(x) for x in range(tot_segments)*samples_per_segment]
        endSamples = [np.int(x) for x in startSamples + samples_per_segment]
        
        
        #generate features for each segment
        mean_data=[]
        std_data=[]
        rms_data=[]
        kurtosis_data=[]
        cov_data=[]
        skew_data=[]
        
        delta_mean=[]
        theta_mean=[]
        alpha_mean=[]
        beta_mean=[]
        all_mean=[]
           
        delta_std=[]
        theta_std=[]
        alpha_std=[]
        beta_std=[]
        all_std=[]
        
        delta_cov=[]
        theta_cov=[]
        alpha_cov=[]
        beta_cov=[]
        all_cov=[]
            
        delta_kurtosis=[]
        theta_kurtosis=[]
        alpha_kurtosis=[]
        beta_kurtosis=[]
        all_kurtosis=[]
        
        delta_skew=[]
        theta_skew=[]
        alpha_skew=[]
        beta_skew=[]
        all_skew=[]
        
        delta_rms=[]
        theta_rms=[]
        alpha_rms=[]
        beta_rms=[]
        all_rms=[]

        delta_data=[]
        theta_data=[]
        alpha_data=[]
        beta_data=[]
        
        for st,en in zip(startSamples,endSamples):
            
            data = welch(EEGdata.loc['traces'][i][st:en], fs=EEGdata.loc['Fs'][i],
                                                    window='hann', nperseg=None, noverlap=None,nfft=None, detrend='constant', return_onesided=True, scaling='density')
            
            # Power spectral density
            Zxx_delta, f1 = bandpass_filter(data[1], data[0],0,4)
            Zxx_theta, f2 = bandpass_filter(data[1], data[0],4,8)
            Zxx_alpha, f3 = bandpass_filter(data[1], data[0],8,12)
            Zxx_beta, f4 = bandpass_filter(data[1], data[0],12,30)
            Zxx_all, f5 = bandpass_filter(data[1], data[0],0,128)
            
            
            # Mean values
            delta_mean.append(np.mean(Zxx_delta))
            theta_mean.append(np.mean(Zxx_theta))
            alpha_mean.append(np.mean(Zxx_alpha))
            beta_mean.append(np.mean(Zxx_beta))
            all_mean.append(np.mean(Zxx_all))
            
            
            # STD values
            delta_std.append(np.std(Zxx_delta))
            theta_std.append(np.std(Zxx_theta))
            alpha_std.append(np.std(Zxx_alpha))
            beta_std.append(np.std(Zxx_beta))
            all_std.append(np.std(Zxx_all))

            # COV values
            delta_cov.append(variation(Zxx_delta))
            theta_cov.append(variation(Zxx_theta))
            alpha_cov.append(variation(Zxx_alpha))
            beta_cov.append(variation(Zxx_beta))
            all_cov.append(variation(Zxx_all))
            
            # Kurtosis values
            delta_kurtosis.append(kurtosis(Zxx_delta))
            theta_kurtosis.append(kurtosis(Zxx_theta))
            alpha_kurtosis.append(kurtosis(Zxx_alpha))
            beta_kurtosis.append(kurtosis(Zxx_beta))
            all_kurtosis.append(kurtosis(Zxx_all))
            
            # Skewness values
            delta_skew.append(skew(Zxx_delta))
            theta_skew.append(skew(Zxx_theta))
            alpha_skew.append(skew(Zxx_alpha))
            beta_skew.append(skew(Zxx_beta))
            all_skew.append(skew(Zxx_all))
            
            # RMS values
            delta_rms.append(np.sqrt(np.trapz(Zxx_delta, f1)))
            theta_rms.append(np.sqrt(np.trapz(Zxx_theta, f2)))
            alpha_rms.append(np.sqrt(np.trapz(Zxx_alpha, f3)))
            beta_rms.append(np.sqrt(np.trapz(Zxx_beta, f4)))
            all_rms.append(np.sqrt(np.trapz(Zxx_all, f5)))
            
            # PSD values
            delta_data.append(bandpower(data[1], data[0],0,4))
            theta_data.append(bandpower(data[1], data[0],4,8))
            alpha_data.append(bandpower(data[1], data[0],8,12))
            beta_data.append(bandpower(data[1], data[0],12,30))
            
            #Time Domain Features
            mean_data.append(np.mean(np.absolute(EEGdata.loc['traces'][i][st:en])))            
            std_data.append(np.std(np.absolute(EEGdata.loc['traces'][i][st:en])))
            kurtosis_data.append(kurtosis(np.absolute(EEGdata.loc['traces'][i][st:en])))
            cov_data.append(variation(np.absolute(EEGdata.loc['traces'][i][st:en])))
            skew_data.append(skew(np.absolute(EEGdata.loc['traces'][i][st:en])))
            rms_data.append(np.sqrt(np.mean((np.absolute(EEGdata.loc['traces'][i][st:en]))**2)))
            
        
        EEGdata.at['delta_mean',i]=delta_mean
        EEGdata.at['theta_mean',i]=theta_mean
        EEGdata.at['alpha_mean',i]=alpha_mean
        EEGdata.at['beta_mean',i]=beta_mean
        EEGdata.at['all_mean',i]=all_mean
        
        EEGdata.at['delta_std',i]=delta_std
        EEGdata.at['theta_std',i]=theta_std
        EEGdata.at['alpha_std',i]=alpha_std
        EEGdata.at['beta_std',i]=beta_std
        EEGdata.at['all_std',i]=all_std
        
        EEGdata.at['delta_cov',i]=delta_cov
        EEGdata.at['theta_cov',i]=theta_cov
        EEGdata.at['alpha_cov',i]=alpha_cov
        EEGdata.at['beta_cov',i]=beta_cov
        EEGdata.at['all_cov',i]=all_cov
            
        EEGdata.at['delta_kurtosis',i]=delta_kurtosis
        EEGdata.at['theta_kurtosis',i]=theta_kurtosis
        EEGdata.at['alpha_kurtosis',i]=alpha_kurtosis
        EEGdata.at['beta_kurtosis',i]=beta_kurtosis
        EEGdata.at['all_kurtosis',i]=all_kurtosis
        
        EEGdata.at['delta_skew',i]=delta_skew
        EEGdata.at['theta_skew',i]=theta_skew
        EEGdata.at['alpha_skew',i]=alpha_skew
        EEGdata.at['beta_skew',i]=beta_skew
        EEGdata.at['all_skew',i]=all_skew
        
        EEGdata.at['delta_rms',i]=delta_rms
        EEGdata.at['theta_rms',i]=theta_rms
        EEGdata.at['alpha_rms',i]=alpha_rms
        EEGdata.at['beta_rms',i]=beta_rms
        EEGdata.at['all_rms',i]=all_rms
        
        EEGdata.at['delta_data',i]=delta_data
        EEGdata.at['theta_data',i]=theta_data
        EEGdata.at['alpha_data',i]=alpha_data
        EEGdata.at['beta_data',i]=beta_data        
        
        #Time domain
        EEGdata.at['mean',i]=mean_data
        EEGdata.at['std',i]=std_data
        EEGdata.at['rms',i]=rms_data
        EEGdata.at['kurtosis',i]=kurtosis_data
        EEGdata.at['cov',i]=cov_data
        EEGdata.at['skewness',i]=skew_data
        
    print('samples_per_segment: {}\nTotal Segment: {}'.format(samples_per_segment,tot_segments))


In [4]:
def butter_bandpass(lowcut, highcut, fs, order=2):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return a, b

In [5]:
def butter_bandpass_filter(data, lowcut, highcut, fs, order=2):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y

In [6]:
def bandpass_filter(Zxx, f, fmin, fmax):
#helper function to calculate bandpower for given freq range
    ind_min = np.argmax(f > fmin) - 1
    ind_max = np.argmax(f > fmax) - 1
    return Zxx[ind_min: ind_max], f[ind_min: ind_max]

In [7]:
def bandpower(Pxx, f, fmin, fmax):
#helper function to calculate bandpower for given freq range    
    ind_min = np.argmax(f > fmin) - 1
    ind_max = np.argmax(f > fmax) - 1
    return np.trapz(Pxx[ind_min: ind_max], f[ind_min: ind_max])

In [ ]:
#the .edf files contain exactly one hour of digitized EEG signals, although those belonging to case chb10 are two hours 
#long, and those belonging to cases chb04, chb06, chb07(case-13: 1Hour), chb09(case-19:2Hour)
# and chb23 are four hours long

#select data directory
dataDir='/home/mirwais/chbmit/chb06'

#create listings of recordings that contain seizures
fileListing=os.listdir(dataDir)
seizureFiles = [x for x in fileListing if 'seizure' in x and '.edf' in x]
normalFiles = [x for x in fileListing if 'seizure' not in x and '.edf' in x]
seizureRecordIDs = [np.int(re.search('(?<=_)(.*?)(?=.edf)',s).group(0).lstrip("0")) for s in seizureFiles]
normalPaths = [os.path.join(dataDir,s) for s in normalFiles]

#aggregate data into df
studyDataList=[]
studyKeyList = []
for idx,files in enumerate(normalPaths):
    #For 1Hour long file: (16/225) => 1 sec,
    #For 2Hours long file: (16/450) => 1 sec,
    #For 4Hours long files: (16/900) => 1 sec
    studyDataList.append(generateEEGdataDF(files,(32/900)))
    studyKeyList.append(np.int(re.search('(?<=_)(.*?)(?=.edf)',normalFiles[idx]).group(0).lstrip("0")))
    
studyDF = pd.concat(studyDataList,axis=0,keys=studyKeyList, names=['recordingID', 'electrode'])
studyDF=studyDF[studyDF.channelName != '-']
studyDF.sort_index(axis=0,level=0,inplace=True,sort_remaining=False)

#label targets
studyDF['sz'] = 0
#for j in seizureRecordIDs:
 #   for idx,st in enumerate(studyDF.index.get_level_values(level=0)):
  #      if(st==j):
   #         studyDF.iloc[idx,-1] = 1
    

#list of derived features to be analyzed
derivedFeatures = studyDF.loc[:,['mean','std','rms','kurtosis','cov','skewness',
                                 'delta_mean','theta_mean','alpha_mean','beta_mean','all_mean',
                                 'delta_std','theta_std','alpha_std','beta_std','all_std',
                                 'delta_cov','theta_cov','alpha_cov','beta_cov','all_cov',
                                 'delta_kurtosis','theta_kurtosis','alpha_kurtosis','beta_kurtosis','all_kurtosis',
                                 'delta_skew','theta_skew','alpha_skew','beta_skew','all_skew',
                                 'delta_rms','theta_rms','alpha_rms','beta_rms','all_rms',
                                 'delta_data','theta_data','alpha_data','beta_data']]

     
#unstack lists into new series
#Tme Domain
t1 = derivedFeatures.apply(lambda x: pd.Series(x['mean']),axis=1).stack().reset_index(level=1, drop=True)
t1.name = 'mean'
t2 = derivedFeatures.apply(lambda x: pd.Series(x['std']),axis=1).stack().reset_index(level=1, drop=True)
t2.name = 'std'
t3 = derivedFeatures.apply(lambda x: pd.Series(x['rms']),axis=1).stack().reset_index(level=1, drop=True)
t3.name='rms'
t4 = derivedFeatures.apply(lambda x: pd.Series(x['kurtosis']),axis=1).stack().reset_index(level=1, drop=True)
t4.name = 'kurtosis'
t5 = derivedFeatures.apply(lambda x: pd.Series(x['cov']),axis=1).stack().reset_index(level=1, drop=True)
t5.name = 'cov'
t6 = derivedFeatures.apply(lambda x: pd.Series(x['skewness']),axis=1).stack().reset_index(level=1, drop=True)
t6.name = 'skewness'

#Freq Domain
f1 = derivedFeatures.apply(lambda x: pd.Series(x['delta_mean']),axis=1).stack().reset_index(level=1, drop=True)
f1.name = 'delta_mean'
f2 = derivedFeatures.apply(lambda x: pd.Series(x['theta_mean']),axis=1).stack().reset_index(level=1, drop=True)
f2.name = 'theta_mean'
f3 = derivedFeatures.apply(lambda x: pd.Series(x['alpha_mean']),axis=1).stack().reset_index(level=1, drop=True)
f3.name = 'alpha_mean'
f4 = derivedFeatures.apply(lambda x: pd.Series(x['beta_mean']),axis=1).stack().reset_index(level=1, drop=True)
f4.name = 'beta_mean'
f5 = derivedFeatures.apply(lambda x: pd.Series(x['all_mean']),axis=1).stack().reset_index(level=1, drop=True)
f5.name = 'all_mean'


f11 = derivedFeatures.apply(lambda x: pd.Series(x['delta_std']),axis=1).stack().reset_index(level=1, drop=True)
f11.name = 'delta_std'
f12 = derivedFeatures.apply(lambda x: pd.Series(x['theta_std']),axis=1).stack().reset_index(level=1, drop=True)
f12.name = 'theta_std'
f13 = derivedFeatures.apply(lambda x: pd.Series(x['alpha_std']),axis=1).stack().reset_index(level=1, drop=True)
f13.name = 'alpha_std'
f14 = derivedFeatures.apply(lambda x: pd.Series(x['beta_std']),axis=1).stack().reset_index(level=1, drop=True)
f14.name = 'beta_std'
f15 = derivedFeatures.apply(lambda x: pd.Series(x['all_std']),axis=1).stack().reset_index(level=1, drop=True)
f15.name = 'all_std'

f16 = derivedFeatures.apply(lambda x: pd.Series(x['delta_cov']),axis=1).stack().reset_index(level=1, drop=True)
f16.name = 'delta_cov'
f17 = derivedFeatures.apply(lambda x: pd.Series(x['theta_cov']),axis=1).stack().reset_index(level=1, drop=True)
f17.name = 'theta_cov'
f18 = derivedFeatures.apply(lambda x: pd.Series(x['alpha_cov']),axis=1).stack().reset_index(level=1, drop=True)
f18.name = 'alpha_cov'
f19 = derivedFeatures.apply(lambda x: pd.Series(x['beta_cov']),axis=1).stack().reset_index(level=1, drop=True)
f19.name = 'beta_cov'
f20 = derivedFeatures.apply(lambda x: pd.Series(x['all_cov']),axis=1).stack().reset_index(level=1, drop=True)
f20.name = 'all_cov'

f21 = derivedFeatures.apply(lambda x: pd.Series(x['delta_kurtosis']),axis=1).stack().reset_index(level=1, drop=True)
f21.name = 'delta_kurtosis'
f22 = derivedFeatures.apply(lambda x: pd.Series(x['theta_kurtosis']),axis=1).stack().reset_index(level=1, drop=True)
f22.name = 'theta_kurtosis'
f23 = derivedFeatures.apply(lambda x: pd.Series(x['alpha_kurtosis']),axis=1).stack().reset_index(level=1, drop=True)
f23.name = 'alpha_kurtosis'
f24 = derivedFeatures.apply(lambda x: pd.Series(x['beta_kurtosis']),axis=1).stack().reset_index(level=1, drop=True)
f24.name = 'beta_kurtosis'
f25 = derivedFeatures.apply(lambda x: pd.Series(x['all_kurtosis']),axis=1).stack().reset_index(level=1, drop=True)
f25.name = 'all_kurtosis'

f26 = derivedFeatures.apply(lambda x: pd.Series(x['delta_skew']),axis=1).stack().reset_index(level=1, drop=True)
f26.name = 'delta_skew'
f27 = derivedFeatures.apply(lambda x: pd.Series(x['theta_skew']),axis=1).stack().reset_index(level=1, drop=True)
f27.name = 'theta_skew'
f28 = derivedFeatures.apply(lambda x: pd.Series(x['alpha_skew']),axis=1).stack().reset_index(level=1, drop=True)
f28.name = 'alpha_skew'
f29 = derivedFeatures.apply(lambda x: pd.Series(x['beta_skew']),axis=1).stack().reset_index(level=1, drop=True)
f29.name = 'beta_skew'
f30 = derivedFeatures.apply(lambda x: pd.Series(x['all_skew']),axis=1).stack().reset_index(level=1, drop=True)
f30.name = 'all_skew'

f31 = derivedFeatures.apply(lambda x: pd.Series(x['delta_rms']),axis=1).stack().reset_index(level=1, drop=True)
f31.name = 'delta_rms'
f32 = derivedFeatures.apply(lambda x: pd.Series(x['theta_rms']),axis=1).stack().reset_index(level=1, drop=True)
f32.name = 'theta_rms'
f33 = derivedFeatures.apply(lambda x: pd.Series(x['alpha_rms']),axis=1).stack().reset_index(level=1, drop=True)
f33.name = 'alpha_rms'
f34 = derivedFeatures.apply(lambda x: pd.Series(x['beta_rms']),axis=1).stack().reset_index(level=1, drop=True)
f34.name = 'beta_rms'
f35 = derivedFeatures.apply(lambda x: pd.Series(x['all_rms']),axis=1).stack().reset_index(level=1, drop=True)
f35.name = 'all_rms'

f36 = derivedFeatures.apply(lambda x: pd.Series(x['delta_data']),axis=1).stack().reset_index(level=1, drop=True)
f36.name = 'delta_data'
f37 = derivedFeatures.apply(lambda x: pd.Series(x['theta_data']),axis=1).stack().reset_index(level=1, drop=True)
f37.name = 'theta_data'
f38 = derivedFeatures.apply(lambda x: pd.Series(x['alpha_data']),axis=1).stack().reset_index(level=1, drop=True)
f38.name = 'alpha_data'
f39 = derivedFeatures.apply(lambda x: pd.Series(x['beta_data']),axis=1).stack().reset_index(level=1, drop=True)
f39.name = 'beta_data'


#concatenate unstacked features
featuresUnstack = pd.concat([t1,t2,t3,t4,t5,t6,f1,f2,f3,f4,f5,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31,f32,f33,f34,f35,f36,f37,f38,f39],axis=1)

#apply targets to reshaped/flattened df
featuresUnstack['sz'] = 0
#for j in seizureRecordIDs:
 #   for idx,st in enumerate(featuresUnstack.index.get_level_values(level=0)):
  #      if (st == j):
   #         featuresUnstack.iloc[idx,-1] = 1

In [ ]:
featuresUnstack.head()

### 4. Storing data in csv file

In [10]:
csvFilePath = '/home/mirwais/notebooks/paper_3/data/chb06_time_freq_segments.csv'

In [11]:
featuresUnstack.to_csv(csvFilePath)

In [12]:
ch = pd.read_csv(csvFilePath)
ch.rename(columns={'Unnamed: 1': 'time'}, inplace=True)

In [14]:
ch.head()

,recordingID,time,mean,std,rms,kurtosis,cov,skewness,delta_mean,theta_mean,...,delta_rms,theta_rms,alpha_rms,beta_rms,all_rms,delta_data,theta_data,alpha_data,beta_data,sz
0,1,0,23.187576,21.627643,31.708337,4.034622,0.932725,1.803416,146.806756,90.185144,...,21.278744,16.019673,10.990561,9.983731,35.010416,452.784950,256.629927,120.792424,99.674887,0
1,1,1,38.768315,32.231725,50.416925,2.855757,0.831393,1.492541,357.289546,161.066688,...,34.554499,21.175390,7.090925,7.342937,47.192678,1194.013383,448.397128,50.281211,53.918730,0
2,1,2,27.248932,22.554292,35.372311,1.026841,0.827713,1.250899,104.014880,95.709603,...,17.959513,17.986306,6.875213,12.370596,32.050453,322.544102,323.507205,47.268555,153.031651,0
3,1,3,35.713523,30.508950,46.970754,1.684626,0.854269,1.334273,365.080520,142.060054,...,33.062310,19.561885,8.404989,9.500706,47.391871,1093.116330,382.667341,70.643834,90.263424,0
4,1,4,32.373321,22.575684,39.467625,1.182820,0.697355,1.057198,118.107324,89.686826,...,19.789520,16.696424,8.422702,9.411599,32.765497,391.625086,278.770563,70.941903,88.578205,0


### 5. Lebaling data b/w normal and seizure

In [15]:
import re
import numpy as np

def extract_data_and_labels(edf_filename, summary_text):
    folder, basename = os.path.split(edf_filename)
    
    i_text_start = summary_text.index(basename)
    
    if 'File Name' in summary_text[i_text_start:]:
        i_text_stop = summary_text.index('File Name', i_text_start)
    else:
        i_text_stop = len(summary_text)
    assert i_text_stop > i_text_start

    file_text = summary_text[i_text_start:i_text_stop]
    if 'Seizure Start' in file_text:
        
        start_sec = int(int(re.search(r"Seizure Start Time: ([0-9]*) seconds", file_text).group(1))/2)
        end_sec = int(int(re.search(r"Seizure End Time: ([0-9]*) seconds", file_text).group(1))/2)
        
        print('Seizure {}:{} => {}'.format(start_sec, end_sec, int(basename[6:8])))
        
        #ch.loc[start_sec:end_sec,'sz']  = ((ch.time >= start_sec) & (ch.time <= end_sec) & (ch.recordingID == int(basename[6:8])))
        ch.loc[((ch.time >= start_sec) & (ch.time <= end_sec) & (ch.recordingID == int(basename[6:8]))), 'sz'] = 2
        #print(ch.loc[start_sec-1:end_sec+1,'sz'])

In [16]:
import glob
import os.path
try:
    file_numbers = [1,2,3]
    for i in file_numbers:
    
        subject_id = 6
        base_path = "/home/mirwais/chbmit/"
        edf_file_names = sorted(glob.glob(os.path.join(base_path, "chb{:02d}/*.edf".format(subject_id))))
        summary_file = os.path.join(base_path, "chb{:02d}/chb{:02d}-{}summary.txt".format(subject_id, subject_id, i))

        summary_content = open(summary_file,'r').read()
        for edf_file_name in edf_file_names:
            result = extract_data_and_labels(edf_file_name, summary_content)
            print(result)
        print('\n.....................................................')
            
except Exception:
    pass

Seizure 862:869 => 1
None
Seizure 163:173 => 4
None
Seizure 6250:6258 => 9
None
Seizure 5416:5422 => 10
None
Seizure 253:259 => 13
None
Seizure 3899:3905 => 18
None
Seizure 4693:4701 => 24
None

.....................................................
Seizure 3730:3738 => 1
None
Seizure 3105:3115 => 4
None
None
None
None
None
None

.....................................................
Seizure 6762:6770 => 1
None
None
None
None
None
None
None

.....................................................


In [23]:
ch[(ch.sz==2) & (ch.recordingID == 24)]

,recordingID,time,mean,std,rms,kurtosis,cov,skewness,delta_mean,theta_mean,...,delta_rms,theta_rms,alpha_rms,beta_rms,all_rms,delta_data,theta_data,alpha_data,beta_data,sz
998270,24,4693,61.617063,43.716265,75.549813,-0.112511,0.709483,0.743614,1029.475970,235.655607,...,56.880340,26.043971,13.145816,15.195145,74.498509,3235.373099,678.288433,172.812481,230.892423,2
998271,24,4694,59.087302,34.133262,68.237737,-0.919625,0.577675,0.162807,916.542320,268.337275,...,52.508143,26.072816,4.700883,7.881020,69.405746,2757.105038,679.791721,22.098297,62.110475,2
998272,24,4695,51.422466,37.535078,63.664371,-0.485780,0.729935,0.678137,660.478725,156.625965,...,46.916772,21.904846,6.451738,9.805022,57.927528,2201.183475,479.822264,41.624917,96.138462,2
998273,24,4696,48.321123,34.233525,59.218791,-0.476957,0.708459,0.622685,481.021886,87.683521,...,38.746644,15.257184,8.349211,8.634962,48.764755,1501.302445,232.781658,69.709323,74.562564,2
998274,24,4697,67.802961,42.864678,80.216097,-0.706598,0.632195,0.352980,1452.523499,85.139119,...,70.589924,15.506033,9.231217,10.490481,80.386307,4982.937343,240.437045,85.215358,110.050188,2
998275,24,4698,46.165293,34.296330,57.510630,1.919886,0.742903,1.040586,417.691006,212.922472,...,33.887901,22.736070,9.774463,11.536240,54.544975,1148.389834,516.928886,95.540127,133.084829,2
998276,24,4699,37.718254,29.687978,48.000445,0.825970,0.787098,1.072378,304.276759,95.268341,...,31.834265,17.059207,6.709074,10.847750,45.375316,1013.420452,291.016549,45.011676,117.673676,2
998277,24,4700,52.862485,36.518117,64.249632,1.543215,0.690813,1.081494,507.311413,403.892767,...,37.613352,33.073574,8.515217,8.186857,62.589724,1414.764255,1093.861288,72.508915,67.024625,2
998278,24,4701,40.010684,27.465207,48.530324,1.401136,0.686447,0.998914,481.307530,102.284974,...,39.202829,15.298591,4.837165,6.844917,49.278565,1536.861802,234.046880,23.398165,46.852891,2
1005470,24,4693,61.719322,53.358847,81.587017,4.207819,0.864540,1.793420,850.206585,355.904254,...,53.776063,33.685308,24.249820,19.309586,78.010233,2891.864950,1134.700005,588.053746,372.860100,2


In [1]:
# Saving data
lebaledCSVFilePath = '/home/mirwais/notebooks/paper_3/data/lebaled/chb06_time_freq_segments.csv'
ch.to_csv(lebaledCSVFilePath)

In [2]:
import pandas as pd
ch = pd.read_csv(lebaledCSVFilePath)
ch.columns.size

44

In [26]:
ch.head()

,Unnamed: 0,recordingID,time,mean,std,rms,kurtosis,cov,skewness,delta_mean,...,delta_rms,theta_rms,alpha_rms,beta_rms,all_rms,delta_data,theta_data,alpha_data,beta_data,sz
0,0,1,0,23.187576,21.627643,31.708337,4.034622,0.932725,1.803416,146.806756,...,21.278744,16.019673,10.990561,9.983731,35.010416,452.784950,256.629927,120.792424,99.674887,0
1,1,1,1,38.768315,32.231725,50.416925,2.855757,0.831393,1.492541,357.289546,...,34.554499,21.175390,7.090925,7.342937,47.192678,1194.013383,448.397128,50.281211,53.918730,0
2,2,1,2,27.248932,22.554292,35.372311,1.026841,0.827713,1.250899,104.014880,...,17.959513,17.986306,6.875213,12.370596,32.050453,322.544102,323.507205,47.268555,153.031651,0
3,3,1,3,35.713523,30.508950,46.970754,1.684626,0.854269,1.334273,365.080520,...,33.062310,19.561885,8.404989,9.500706,47.391871,1093.116330,382.667341,70.643834,90.263424,0
4,4,1,4,32.373321,22.575684,39.467625,1.182820,0.697355,1.057198,118.107324,...,19.789520,16.696424,8.422702,9.411599,32.765497,391.625086,278.770563,70.941903,88.578205,0


In [28]:
############################## End ###########################################################